In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

import tensorflow as tf
from tensorflow.compat.v1.keras.backend import set_session
from tensorflow.keras import backend as K
config = tf.compat.v1.ConfigProto()
# A "Best-fit with coalescing" algorithm, simplified from a version of dlmalloc.
config.gpu_options.allocator_type = 'BFC' 
config.gpu_options.allow_growth = True
config.allow_soft_placement = True
set_session(tf.compat.v1.Session(config=config))

In [3]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from keras.datasets import imdb

Using TensorFlow backend.


In [4]:
MAX_NUM_WORDS = 10000 # 词表的大小
MAX_LEN = 300 # 每条文本的最多词数

In [5]:
# 两句np函数防止报错
np.load.__defaults__=(None, True, True, 'ASCII')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=MAX_NUM_WORDS)
np.load.__defaults__=(None, False, True, 'ASCII')

In [6]:
print("Train entries: {}, labels: {}".format(len(x_train), len(y_train)))
print("Test entries: {}, labels: {}".format(len(x_test), len(y_test)))
len(x_train[0]), len(x_train[1])

Train entries: 25000, labels: 25000
Test entries: 25000, labels: 25000


(218, 189)

In [7]:
# 将输出 one hot 化 （0，1，0，0）
num_classes = 2
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(x_train[0], y_train[1])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32] [1. 0.]


In [8]:
# IMDB的单词映射表，key是单词，value是索引
word_index = imdb.get_word_index()

# 第一个key是空的
word_index = {k:(v+3) for k,v in word_index.items()} 
word_index["<PAD>"] = 0 # 填充单词
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # 未知单词
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
print(reverse_word_index[4])

#把数字序列转化为相应的字符串
def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

the


In [11]:
from tensorflow.keras.preprocessing import sequence

x_train = sequence.pad_sequences(x_train, maxlen=MAX_LEN, padding='post')
x_test = sequence.pad_sequences(x_test, maxlen=MAX_LEN, padding='post')
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
decode_review(x_train[2])

x_train shape: (25000, 300)
x_test shape: (25000, 300)


"<START> this has to be one of the worst films of the 1990s when my friends i were watching this film being the target audience it was aimed at we just sat watched the first half an hour with our jaws touching the floor at how bad it really was the rest of the time everyone else in the theatre just started talking to each other leaving or generally crying into their popcorn that they actually paid money they had <UNK> working to watch this feeble excuse for a film it must have looked like a great idea on paper but on film it looks like no one in the film has a clue what is going on crap acting crap costumes i can't get across how <UNK> this is to watch save yourself an hour a bit of your life <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD

In [12]:
EMBED_SIZE = 128

char_input = keras.Input(shape=(MAX_LEN,), name="input_char")
embed = layers.Embedding(input_dim=MAX_NUM_WORDS+1, 
                         output_dim=EMBED_SIZE, 
                         input_length=MAX_LEN,
                         trainable=True)(char_input)
# 卷积层和池化层，设置卷积核大小分别为3,4,5
cnn1 = layers.Conv1D(256, 3, padding='same', strides=1, activation='relu')(embed)
cnn1 = layers.MaxPooling1D(pool_size=298)(cnn1)
cnn2 = layers.Conv1D(256, 4, padding='same', strides=1, activation='relu')(embed)
cnn2 = layers.MaxPooling1D(pool_size=297)(cnn2)
cnn3 = layers.Conv1D(256, 5, padding='same', strides=1, activation='relu')(embed)
cnn3 = layers.MaxPooling1D(pool_size=296)(cnn3)
# 合并三个模型的输出向量
cnn = K.concatenate([cnn1, cnn2, cnn3], axis=-1)
flat = layers.Flatten()(cnn)
drop = layers.Dropout(0.2)(flat) #在池化层到全连接层之前可以加上dropout防止过拟合
output_layer = layers.Dense(2, activation='sigmoid')(drop)

In [13]:
from tensorflow.keras.models import *
model = Model(inputs=char_input, outputs=output_layer)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_char (InputLayer)         [(None, 300)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 300, 128)     1280128     input_char[0][0]                 
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 300, 256)     98560       embedding[0][0]                  
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 300, 256)     131328      embedding[0][0]                  
______________________________________________________________________________________________

In [14]:
batch_size = 1000
epochs = 5

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 15s 775us/sample - loss: 0.6822 - accuracy: 0.5865 - val_loss: 0.6586 - val_accuracy: 0.7277
Epoch 2/5
20000/20000 [==============================] - 12s 579us/sample - loss: 0.5960 - accuracy: 0.7792 - val_loss: 0.5055 - val_accuracy: 0.8060
Epoch 3/5
20000/20000 [==============================] - 11s 568us/sample - loss: 0.4098 - accuracy: 0.8300 - val_loss: 0.3420 - val_accuracy: 0.8521
Epoch 4/5
20000/20000 [==============================] - 11s 557us/sample - loss: 0.2782 - accuracy: 0.8869 - val_loss: 0.2906 - val_accuracy: 0.8852
Epoch 5/5
20000/20000 [==============================] - 12s 584us/sample - loss: 0.2036 - accuracy: 0.9227 - val_loss: 0.2792 - val_accuracy: 0.8903


In [15]:
score = model.evaluate(x_test, y_test, verbose=0)
print("测试集 loss:", score[0])
print("测试集准确度", score[1])

测试集 loss: 0.2871262518119812
测试集准确度 0.87974


In [16]:
y_pred = model.predict(x_test, batch_size=1000)

In [17]:
def show_result(idx):
    text = decode_review(x_test[idx])
    pred_result = np.argmax(y_pred[idx])
    true_result = np.argmax(y_test[idx])
    result_list = ["negative", "positive"]
    print("text: {}\npred:{}\ntrue:{}".format(text, result_list[pred_result], result_list[true_result]))

show_result(3)

text: <START> i generally love this type of movie however this time i found myself wanting to kick the screen since i can't do that i will just complain about it this was absolutely idiotic the things that happen with the dead kids are very cool but the alive people are absolute idiots i am a grown man pretty big and i can defend myself well however i would not do half the stuff the little girl does in this movie also the mother in this movie is reckless with her children to the point of neglect i wish i wasn't so angry about her and her actions because i would have otherwise enjoyed the flick what a number she was take my advise and fast forward through everything you see her do until the end also is anyone else getting sick of watching movies that are filmed so dark anymore one can hardly see what is being filmed as an audience we are <UNK> involved with the actions on the screen so then why the hell can't we have night vision <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PA